# Food Delivery Dataset Analysis (Hackathon)

## Objective
We are given 3 datasets:
- orders.csv (Transactional Data)
- users.json (User Master Data)
- restaurants.sql (Restaurant Master Data)

We will merge them using LEFT JOIN:
- orders.user_id → users.user_id
- orders.restaurant_id → restaurants.restaurant_id

Final output: final_food_delivery_dataset.csv


In [14]:
import pandas as pd
import json
import sqlite3


In [15]:
orders = pd.read_csv("orders.csv")
print("orders.csv shape:", orders.shape)
orders.head()


orders.csv shape: (10000, 6)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [16]:
with open("users.json", "r", encoding="utf-8") as f:
    users_data = json.load(f)

users = pd.json_normalize(users_data)

print("users.json shape:", users.shape)
users.head()


users.json shape: (3000, 4)


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [17]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

cursor.executescript(sql_script)

restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)

print("restaurants table shape:", restaurants.shape)
restaurants.head()


restaurants table shape: (500, 4)


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [18]:
final_df = orders.merge(users, on="user_id", how="left")
final_df = final_df.merge(restaurants, on="restaurant_id", how="left")

print("Final merged dataset shape:", final_df.shape)
final_df.head()


Final merged dataset shape: (10000, 12)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [19]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)
print("✅ Saved: final_food_delivery_dataset.csv")


✅ Saved: final_food_delivery_dataset.csv


In [20]:
# Total orders placed by Gold members
gold_orders_count = final_df[final_df["membership"] == "Gold"].shape[0]

# Total revenue generated from Hyderabad (rounded)
hyderabad_total_revenue = round(final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum())

# Distinct users placed at least one order
distinct_users = final_df["user_id"].nunique()

# Average order value for Gold members (2 decimals)
gold_avg_order_value = round(final_df[final_df["membership"] == "Gold"]["total_amount"].mean(), 2)

# Orders for restaurants with rating >= 4.5
orders_rating_45 = final_df[final_df["rating"] >= 4.5].shape[0]

# Orders in top revenue city among Gold members only
gold_df = final_df[final_df["membership"] == "Gold"]
gold_city_revenue = gold_df.groupby("city")["total_amount"].sum().sort_values(ascending=False)

top_gold_city = gold_city_revenue.index[0]
gold_orders_top_city = gold_df[gold_df["city"] == top_gold_city].shape[0]

print("✅ Total Gold orders:", gold_orders_count)
print("✅ Hyderabad total revenue:", hyderabad_total_revenue)
print("✅ Distinct users:", distinct_users)
print("✅ Gold average order value:", gold_avg_order_value)
print("✅ Orders rating >= 4.5:", orders_rating_45)
print("✅ Top Gold revenue city:", top_gold_city)
print("✅ Orders in that city (Gold only):", gold_orders_top_city)


✅ Total Gold orders: 4987
✅ Hyderabad total revenue: 1889367
✅ Distinct users: 2883
✅ Gold average order value: 797.15
✅ Orders rating >= 4.5: 3374
✅ Top Gold revenue city: Chennai
✅ Orders in that city (Gold only): 1337


## Fill in the blanks (Dataset Understanding)

1) The column used to join orders.csv and users.json is **user_id**.  
2) The dataset containing cuisine and rating information is stored in **SQL (.sql)** format.  
3) The total number of rows in the final merged dataset is **10000**.  
4) If a user has no matching record in users.json, the merged values will be **NaN (missing values)**.  
5) The Pandas function used to combine datasets based on a key is **merge()**.  
6) The column membership in the final dataset originates from the **users.json** file.  
7) The join key used to combine orders data with restaurant details is **restaurant_id**.  
8) The column that helps identify the type of food served by a restaurant is **cuisine**.  
9) If a user places multiple orders, their personal details appear **multiple times (once per order)**.
